In [1]:
from mysql.connector import MySQLConnection, Error
import pandas as pd
from sqlalchemy import create_engine 
import pymysql
import datetime
import pytz
from fbprophet import Prophet
from sklearn.metrics import mean_absolute_error
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from connection_config import read_db_config
from fbprophet.diagnostics import cross_validation
from sklearn.model_selection import ParameterGrid


In [2]:
conn_string,ssl_args = read_db_config('teama.teama')
engine = create_engine(conn_string,connect_args=ssl_args)
clean_load_2020= pd.read_sql('SELECT * FROM totalload_2020_entsoe_clean', con=engine,index_col = 'index')
clean_load_2019= pd.read_sql('SELECT * FROM totalload_2019_entsoe_clean', con=engine,index_col = 'index')
clean_load_2015_2018= pd.read_sql('SELECT * FROM totalload_2015_2018_entsoe_clean', con=engine,index_col = 'index')

In [3]:
clean_load_2015_2019 = pd.concat([clean_load_2015_2018,clean_load_2019]).reset_index()
clean_load_2015_2019 = clean_load_2015_2019.drop(["index"],axis=1)

## Starting my baseline model

In [4]:
n = Prophet(daily_seasonality=False)
n.add_country_holidays(country_name='UK')
n.fit(clean_load_2015_2019)

future = n.make_future_dataframe(periods=365, freq='D')
forecast = n.predict(future)

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [5]:
df_cv = cross_validation(n,initial = '1095 days',period='180 days',horizon ='365 days')

INFO:fbprophet:Making 3 forecasts with cutoffs between 2018-01-05 00:00:00 and 2018-12-31 00:00:00


In [6]:
mae_baseline = mean_absolute_error(df_cv.y,df_cv.yhat)
print('MAE: %.3f' % mae_baseline)

MAE: 1377.359


## See how much holidays affect my model

In [10]:
params_grid = {'growth' : ["linear"],'holidays_prior_scale':[10, 20, 30, 40]}
grid = ParameterGrid(params_grid)
print([p for p in grid])

[{'growth': 'linear', 'holidays_prior_scale': 10}, {'growth': 'linear', 'holidays_prior_scale': 20}, {'growth': 'linear', 'holidays_prior_scale': 30}, {'growth': 'linear', 'holidays_prior_scale': 40}]


In [11]:
mae_holiday=[]
for p in grid:
    m =Prophet(**p)
    m.add_country_holidays(country_name='UK')
    m.fit(clean_load_2015_2019)
    df_cv = cross_validation(m,initial = '1095 days',period='180 days',horizon ='365 days')
    mae_baseline = mean_absolute_error(df_cv.y,df_cv.yhat)
    mae_holiday.append(mae_baseline)
    #do scoring or diag
    #save p and score

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 3 forecasts with cutoffs between 2018-01-05 00:00:00 and 2018-12-31 00:00:00


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


INFO:fbprophet:Making 3 forecasts with cutoffs between 2018-01-05 00:00:00 and 2018-12-31 00:00:00


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


INFO:fbprophet:Making 3 forecasts with cutoffs between 2018-01-05 00:00:00 and 2018-12-31 00:00:00


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


INFO:fbprophet:Making 3 forecasts with cutoffs between 2018-01-05 00:00:00 and 2018-12-31 00:00:00


In [12]:
mae_holiday

[1377.3592025433893,
 1360.2395617847799,
 1369.9350750373058,
 1384.3407579181746]

### holidays_prior_scale seems the best at 20

In [14]:
params_grid_season = {'growth' : ["linear"],'holidays_prior_scale':[20],'seasonality_mode':['additive', 'multiplicative']}
grid = ParameterGrid(params_grid_season)
print([p for p in grid])

[{'growth': 'linear', 'holidays_prior_scale': 20, 'seasonality_mode': 'additive'}, {'growth': 'linear', 'holidays_prior_scale': 20, 'seasonality_mode': 'multiplicative'}]


In [15]:
mae_season=[]
for p in grid:
    m =Prophet(**p)
    m.add_country_holidays(country_name='UK')
    m.fit(clean_load_2015_2019)
    df_cv = cross_validation(m,initial = '1095 days',period='180 days',horizon ='365 days')
    mae_baseline = mean_absolute_error(df_cv.y,df_cv.yhat)
    mae_season.append(mae_baseline)
    
mae_season

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 3 forecasts with cutoffs between 2018-01-05 00:00:00 and 2018-12-31 00:00:00


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


INFO:fbprophet:Making 3 forecasts with cutoffs between 2018-01-05 00:00:00 and 2018-12-31 00:00:00


[1360.2395617847799, 1438.6570036045587]

In [29]:
m = Prophet(daily_seasonality=False, growth = "linear",holidays_prior_scale=20,seasonality_mode='additive')
m.add_country_holidays(country_name='UK')
m.fit(clean_load_2015_2019)

In [30]:
future_2020 = m.make_future_dataframe(periods=clean_load_2020.shape[0], freq='D')
forecast_2020 = m.predict(future_2020)
forecast_2020_short = forecast_2020[-clean_load_2020.shape[0]:]

fig = go.Figure()
fig.add_trace(go.Scatter(x=clean_load_2020['ds'], y=clean_load_2020['y'],
                    mode='lines',
                    name='Actual'))
fig.add_trace(go.Scatter(x=clean_load_2020['ds'], y=forecast_2020_short['yhat'],
                    mode='lines',
                    name='Predicted'))
fig.update_layout(height=500, width=900, title_text="Actual load in 2020 vs. predicted without Covid")
fig.show()

In [32]:
date_load_daily_2015_2020 = pd.concat([clean_load_2015_2019, clean_load_2020], axis=0)
# Compare to truth on whole timescale
fig = go.Figure()
fig.add_trace(go.Scatter(x=date_load_daily_2015_2020['ds'], y=date_load_daily_2015_2020['y'],
                    mode='lines',
                    name='Actual'))
fig.add_trace(go.Scatter(x=date_load_daily_2015_2020['ds'], y=forecast_2020['yhat'],
                    mode='lines',
                    name='Predicted'))
fig.update_layout(height=500, width=900, title_text="Actual load 2015-2020 vs. predicted without Covid")
fig.show()

In [43]:
model_result_2015_2020 = pd.DataFrame({'Timestamp':date_load_daily_2015_2020['ds'].values,'Predicted':forecast_2020['yhat'].values,'Actual':date_load_daily_2015_2020['y'].values})
model_result_2015_2020.to_sql('model_only_load_2015_2020_tuned_trained_2015_2019',con = engine,if_exists = 'fail')